In [ ]:
import numpy as np
import pandas as pd
import tqdm
import gensim
import nltk
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

# !wget "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

In [ ]:
train = pd.read_csv("../input/commonlitreadabilityprize/train.csv")
test = pd.read_csv('../input/commonlitreadabilityprize/test.csv')
test = test['excerpt']

text_token = []
stemmer = PorterStemmer()
for text in train['excerpt']:
    word_token = []
    sentences = sent_tokenize(text)
    for sentence in sentences:
        words = word_tokenize(sentence)
        for word in words:
            if word not in stopwords.words('english'):
                word = stemmer.stem(word)
                word_token.append(word)
    text_token.append(word_token)
    
test_text_token = []
stemmer = PorterStemmer()
for text in test:
    word_token = []
    sentences = sent_tokenize(text)
    for sentence in sentences:
        words = word_tokenize(sentence)
        for word in words:
            if word not in stopwords.words('english'):
                word = stemmer.stem(word)
                word_token.append(word)
    test_text_token.append(word_token)

In [ ]:
word2vec_model = gensim.models.KeyedVectors.load_word2vec_format('../input/googlenewsvectorsnegative300/GoogleNews-vectors-negative300.bin', binary=True)
print(word2vec_model.vectors.shape)
# from gensim.models import KeyedVectors
# model.wv.save_word2vec_format('eng_w2v')
# !python -m gensim.scripts.word2vec2tensor --input eng_w2v --output eng_w2v

def get_vector(word):
    if word in word2vec_model:
        return word2vec_model[word]
    else:
        return None

words = list(word2vec_model.index_to_key)
for i in tqdm.tqdm(range(len(text_token))):
    embedding_matrix = np.zeros((216, 300))
    for j in range(len(text_token[i])):
        temp = get_vector(text_token[i][j])
        if temp is not None:
            embedding_matrix[j] = temp
    text_token[i] = embedding_matrix
    
words = list(word2vec_model.index_to_key)
for i in tqdm.tqdm(range(len(test_text_token))):
    embedding_matrix = np.zeros((216, 300))
    for j in range(len(test_text_token[i])):
        temp = get_vector(test_text_token[i][j])
        if temp is not None:
            embedding_matrix[j] = temp
    test_text_token[i] = embedding_matrix

In [ ]:
X = np.array(text_token)
y = train['target']
X_test =np.array(test_text_token)

from keras.models import Sequential
from keras.layers import Dense, LSTM, InputLayer, Bidirectional,Dropout
from keras.optimizers import Adam

model = Sequential()
model.add(Bidirectional(LSTM(256, return_sequences=False), input_shape=X.shape[1:]))
model.add(Dense(100))
model.add(Dropout(0.25))
model.add(Dense(10))
model.add(Dense(1))
model.compile(loss='mean_squared_error',optimizer=Adam(0.001))

model.summary()

In [ ]:
model.fit(X,y,validation_split=0.4, epochs=10, batch_size=256,verbose=1)

In [ ]:
predict = model.predict(X_test)
submit = pd.read_csv('../input/commonlitreadabilityprize/sample_submission.csv')
submit['target'] = predict
submit.to_csv('submission.csv',index=None)